# TSA Chapter 3: Deterministic vs Stochastic Trends

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_Ch3/TSA_ch3_trend_comparison/TSA_ch3_trend_comparison.ipynb)

This notebook demonstrates:
- Key distinction: deterministic trends removed by detrending, stochastic trends (unit roots) removed by differencing.


In [ ]:
!pip install matplotlib numpy scipy statsmodels pandas -q

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox

# Style configuration
COLORS = {
    'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32',
    'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD',
}

plt.rcParams.update({
    'axes.facecolor': 'none', 'figure.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False,
    'axes.spines.right': False, 'axes.grid': False, 'font.size': 9,
    'axes.titlesize': 10, 'axes.labelsize': 9, 'xtick.labelsize': 8,
    'ytick.labelsize': 8, 'legend.fontsize': 8, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.edgecolor': '#333333', 'axes.linewidth': 0.8,
})

np.random.seed(42)

def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}.pdf + .png')

In [ ]:
n = 300
t = np.arange(n)
eps = np.random.normal(0, 1, n)

# Deterministic trend
det_trend = 0.1 * t + eps

# Stochastic trend (random walk with drift)
stoch_trend = np.cumsum(eps + 0.1)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Original series
axes[0, 0].plot(det_trend, color='#1A3A6E', linewidth=1)
axes[0, 0].plot(0.1 * t, color='red', linestyle='--', linewidth=1.5, label='Trend: 0.1t')
axes[0, 0].set_title('Deterministic Trend', fontweight='bold')
axes[0, 0].legend()

axes[0, 1].plot(stoch_trend, color='#DC3545', linewidth=1)
axes[0, 1].set_title('Stochastic Trend (RW + Drift)', fontweight='bold')

# After treatment
detrended = det_trend - 0.1 * t
differenced = np.diff(stoch_trend)

adf1 = adfuller(detrended)[1]
adf2 = adfuller(differenced)[1]

axes[1, 0].plot(detrended, color='#2E7D32', linewidth=0.8)
axes[1, 0].axhline(0, color='red', linestyle='--', linewidth=0.5)
axes[1, 0].set_title(f'After Detrending (ADF p={adf1:.4f})', fontweight='bold')
axes[1, 0].set_xlabel('Time')

axes[1, 1].plot(differenced, color='#2E7D32', linewidth=0.8)
axes[1, 1].axhline(0, color='red', linestyle='--', linewidth=0.5)
axes[1, 1].set_title(f'After Differencing (ADF p={adf2:.4f})', fontweight='bold')
axes[1, 1].set_xlabel('Time')

plt.suptitle('Deterministic vs Stochastic Trends', fontweight='bold', fontsize=13)
plt.tight_layout()
save_chart(fig, 'ch3_trend_comparison')
plt.show()

print("Key Distinction:")
print("  Deterministic trend: Remove by detrending (regression)")
print("  Stochastic trend: Remove by differencing")
print("  Wrong method can lead to incorrect inference!")